##Fine-Tunning models
North Centrar US

gpt-35-turbo (0613)
davinci-002
babbage-002

#### Set Up

In [ ]:
import os
import openai
import datetime
import json
import numpy as np
import requests
import sys
import time
import tiktoken

from dotenv import load_dotenv



openai.api_type: str = "azure"
openai.api_key = os.getenv("AZURE_OAI_KEY_NW")
openai.api_base = os.getenv("AZURE_OAI_ENDPOINT_NW")
openai.api_version = "2023-09-15-preview"

In [ ]:
model = "gpt-35-turbo-16k"

### Funciones Básicas

In [ ]:
def get_chat_completion(prompt, model=model, max_tokens=200):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, 
        max_tokens = max_tokens,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

def get_chat_completioncomp(prompt, model=model, max_tokens=200):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, 
        max_tokens = max_tokens,
        top_p = 1.0
    )
    return response

In [ ]:
response=get_chat_completioncomp("Hola")
print(response)

In [ ]:
print("Answer type:\n\n", type(response.choices[0].message))

### Entrenamiento

In [ ]:
training_file_name = "3.13 entrenamiento.jsonl"
validation_file_name = "3.13 validacion.jsonl"

In [ ]:
with open(training_file_name, "r") as f:
    print(f.read())

In [ ]:

with open(validation_file_name, "r") as f:
    print(f.read())

### Fine tuning

In [ ]:
# Cargamos los datos del json en una lista
with open(training_file_name, "r", encoding="utf-8") as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Número de ejemplos de la lista:", len(training_dataset))
print("Primer ejemplo de la lista:")
for message in training_dataset[0]["messages"]:
    print(message)

In [ ]:
# Cargamos los datos de validación
with open(validation_file_name, "r", encoding="utf-8") as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNúmero de ejemplos de la lista:", len(validation_dataset))
print("Primer ejemplo de la lista:")
for message in validation_dataset[0]["messages"]:
    print(message)

## Información de tokens

In [ ]:
encoding = tiktoken.get_encoding("cl100k_base")

In [ ]:
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    """
    Number of tokens from messages
    """
    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name

    num_tokens += 3

    return num_tokens

In [ ]:
def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))

    return num_tokens

In [ ]:
files = [training_file_name, validation_file_name]

for file in files:
    print()
    print("-" * 100)
    print(f"Processing file: {file}")
    print("-" * 100)

    with open(file, "r", encoding="utf-8") as f:
        dataset = [json.loads(line) for line in f]

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

In [ ]:
total_tokens, assistant_tokens

Ejecutando el proceso de Fine Tuning

In [ ]:
training_response = openai.File.create(
    file=open(training_file_name, "rb"),
    purpose="fine-tune",
    user_provided_filename=training_file_name,
)
training_file_id = training_response["id"]


validation_response = openai.File.create(
    file=open(validation_file_name, "rb"),
    purpose="fine-tune",
    user_provided_filename=validation_file_name,
)
validation_file_id = validation_response["id"]

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

In [ ]:
ftmodel = "gpt-35-turbo-0613"

#gpt-35-turbo (0613)

In [ ]:
response = openai.FineTune.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=ftmodel,
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])
print(response)